In [90]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

inc_updrs_sigfall = pd.read_csv('./working_data/normalised_increase_updrs_sigfall.csv')

def build_confusion_matrix(dataset, model):
        X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.25)
        predictions = model.fit(X_train, y_train).predict(X_test)
        tn, fn, fp, tp = confusion_matrix(predictions, y_test.values).ravel()
        return {'true_negative': tn, 'false_negative': fn, 'false_positive': fp, 'true_positive': tp}

def sensitivity(cm):
    return cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])

def specificity(cm):
    return cm['true_negative'] / ( cm['true_negative'] + cm['false_positive'] )

def false_positive_rate(cm):
    return (1.0 - (specificity(cm)) )

def generate_scores(dataset, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        cm = build_confusion_matrix(dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }


In [ ]:
# ------------ inc_updrs_sigfall data ----------------

In [92]:
# Create a model for each of the 3 solvers

# Solver 1 : Singular value decomposition
svd_scores = generate_scores(inc_updrs_sigfall, LinearDiscriminantAnalysis(solver="svd"))

# Solver 2 : Least squares solution
lsqr_scores = generate_scores(inc_updrs_sigfall, LinearDiscriminantAnalysis(solver="lsqr"))

# Solver 3 : Eigenvalue decomposition
eigen_scores = generate_scores(inc_updrs_sigfall, LinearDiscriminantAnalysis(solver="eigen"))

In [98]:
print('--- eigen ---')
print(eigen_scores)
print('\n--- lsqr ---')
print(lsqr_scores)
print('\n--- svd ---')
print(svd_scores)

--- eigen ---
{'specificity': 0.9779883259112319, 'sensitivity': 0.11967697968324059}

--- lsqr ---
{'specificity': 0.9783092911651454, 'sensitivity': 0.12068654609236211}

--- svd ---
{'specificity': 0.9779469427828729, 'sensitivity': 0.12066162632706832}


In [102]:

# ------------ inc_updrs_sigfall_raw data ----------------

In [99]:
inc_updrs_sigfall_raw = pd.read_csv('./working_data/normalised_increase_updrs_sigfall_raw.csv')



# Solver 1 : Singular value decomposition
svd_scores_raw = generate_scores(inc_updrs_sigfall_raw, LinearDiscriminantAnalysis(solver="svd"))

# Solver 2 : Least squares solution
lsqr_scores_raw = generate_scores(inc_updrs_sigfall_raw, LinearDiscriminantAnalysis(solver="lsqr"))

# Solver 3 : Eigenvalue decomposition
eigen_scores_raw = generate_scores(inc_updrs_sigfall_raw, LinearDiscriminantAnalysis(solver="eigen"))

In [101]:
print('--- eigen ---')
print(eigen_scores_raw)
print('\n--- lsqr ---')
print(lsqr_scores_raw)
print('\n--- svd ---')
print(svd_scores_raw)


--- eigen ---
{'specificity': 0.9792370559936376, 'sensitivity': 0.11323439981334102}

--- lsqr ---
{'specificity': 0.9787811376469111, 'sensitivity': 0.1141919351809565}

--- svd ---
{'specificity': 0.9795701965004964, 'sensitivity': 0.11433337281537215}
